In [11]:
from pydub.exceptions import CouldntDecodeError
import os
from pydub import AudioSegment
import pandas as pd

rootdir = '../EMBRACE-data-analysis/data/summer23-pitt'

df_duration_audios = []#pd.DataFrame()

for subdir, dirs, files in os.walk(rootdir):
    total_time_duration = 0
    for file in files:
        file_path = os.path.join(subdir, file)
        #print(file_path)
        if file_path.endswith(".mp3"):
            try:
                audio = AudioSegment.from_file(file_path)
                total_time_duration = total_time_duration + audio.duration_seconds
            except CouldntDecodeError:
                print("Error on reading the file: "+file_path)

    print(subdir+" -> total time duration: "+str(total_time_duration)+" seconds")
    if(total_time_duration>0):
        df_duration_audios.append(total_time_duration)

#show stats about total duration of the audios per family
sum_durations = 0
total_families = 0
for i in range(0,len(df_duration_audios)):
    print(df_duration_audios[i])
    total_families = total_families + 1
    sum_durations = sum_durations + df_duration_audios[i]
print("average of total duration is: ")
print(sum_durations/total_families)

#print(df_duration_audios.describe())




../EMBRACE-data-analysis/data/summer23-pitt -> total time duration: 0 seconds
../EMBRACE-data-analysis/data/summer23-pitt/Family_05 -> total time duration: 0 seconds
../EMBRACE-data-analysis/data/summer23-pitt/Family_05/05_Chromebook Data -> total time duration: 0 seconds
Error on reading the file: ../EMBRACE-data-analysis/data/summer23-pitt/Family_05/05_Chromebook Data/05_voice-recordings/voice-recording-1683222907231.mp3
../EMBRACE-data-analysis/data/summer23-pitt/Family_05/05_Chromebook Data/05_voice-recordings -> total time duration: 8857.944000000001 seconds
../EMBRACE-data-analysis/data/summer23-pitt/Family_05/05_Chromebook Data/05_data -> total time duration: 0 seconds
../EMBRACE-data-analysis/data/summer23-pitt/Family_05/05_Chromebook Data/05_data/journal -> total time duration: 0 seconds
../EMBRACE-data-analysis/data/summer23-pitt/Family_05/05_Chromebook Data/05_data/diagnostic.data -> total time duration: 0 seconds
../EMBRACE-data-analysis/data/summer23-pitt/Family_02 -> tota